In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, isnan, count, udf, round as spark_round
from pyspark.sql.types import StringType, FloatType


from src.utils.s3_manager import S3Manager
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os 
import nltk


In [2]:
data_dir = "/Users/ilan/big-data-airflow-project/data"

In [3]:
spark = SparkSession.builder \
    .appName("EDA with Spark") \
    .getOrCreate()

24/05/21 20:35:17 WARN Utils: Your hostname, Ordinateur-portable-de-Ilan.local resolves to a loopback address: 127.0.0.1; using 192.168.1.18 instead (on interface en0)
24/05/21 20:35:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/21 20:35:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
df = spark.read.parquet(data_dir+"/allocine_movies.parquet", header=True, inferSchema=True)

In [13]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Genre: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Actors: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Action: integer (nullable = true)
 |-- Adventure: integer (nullable = true)
 |-- Animation: integer (nullable = true)
 |-- Biopic: integer (nullable = true)
 |-- Comedy: integer (nullable = true)
 |-- Crime: integer (nullable = true)
 |-- Documentary: integer (nullable = true)
 |-- Drama: integer (nullable = true)
 |-- Horror: integer (nullable = true)
 |-- Famille: integer (nullable = true)
 |-- Guerre: integer (nullable = true)
 |-- MusicalRomance: integer (nullable = true)
 |-- Sci-Fi: integer (nullable = true)
 |-- Thriller: integer (nullable = true)
 |-- Western: integer (nullable = true)


In [ ]:
df.show()

In [ ]:
print("The shape of the allocine dataset is ", (df.count(), len(df.columns)))

In [ ]:
df = df.drop("Release Date", "Director")

# Renaming columns to match the netflix dataset

In [ ]:
df = df.withColumnRenamed("Duration", "Runtime")
df = df.withColumnRenamed("Synopsis", "Summary")

# Dealing with missing values

In [ ]:
missing_values = df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns])
missing_values.show()

#### Runtime column

In [ ]:
df = df.dropna(subset=["Runtime"])

In [ ]:
def convert_runtime_to_interval(runtime):
    hours, minutes = map(int, runtime.replace('min', '').replace('h', '').split())
    total_hours = hours + minutes / 60
    if total_hours > 2:
        return '> 2 hrs'
    elif total_hours < 0.5:
        return '< 30 minutes'
    elif total_hours < 1 and total_hours >= 0.5 :
        return '30 - 60 mins'
    else:
        return '1-2 hour'
    

In [ ]:
convert_runtime_udf = udf(convert_runtime_to_interval, StringType())

In [ ]:
df = df.withColumn("Runtime", convert_runtime_udf(df["Runtime"]))

In [ ]:
df.show()

#### Rating column

We're gonna merge the spectator rating and the press rating into one column called rating

In [ ]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("Press Rating", regexp_replace(col("Press Rating"), ",", "."))
df = df.withColumn("Press Rating", when(col("Press Rating") == "--", None).otherwise(col("Press Rating")))
df = df.withColumn("Press Rating", col("Press Rating").cast(FloatType()))

df = df.withColumn("Spectator Rating", regexp_replace(col("Spectator Rating"), ",", "."))
df = df.withColumn("Spectator Rating", when(col("Spectator Rating") == "--", None).otherwise(col("Spectator Rating")))
df = df.withColumn("Spectator Rating", col("Spectator Rating").cast(FloatType()))

In [ ]:
from pyspark.sql.functions import mean

mean_press = df.select(mean(col("Press Rating")).alias("mean_press")).collect()[0]["mean_press"]
df = df.na.fill({"Press Rating": mean_press})

mean_spectator = df.select(mean(col("Spectator Rating")).alias("mean_spectator")).collect()[0]["mean_spectator"]
df = df.na.fill({"Spectator Rating": mean_spectator})

In [ ]:
df = df.withColumn("Rating", spark_round((col("Press Rating") + col("Spectator Rating")) / 2, 1))
df = df.drop("Press Rating", "Spectator Rating")

In [ ]:
df.show()

#### Transform the genre column. We will create one col for each genre and fill it with 1 if the movie belongs to this genre, 0 otherwise

In [ ]:
from pyspark.sql.functions import split

df = df.withColumn("Genre", split(col("Genre"), ", "))
genres = ["Action", "Adventure", "Drama", "Sci-Fi", "Crime", "Thriller", "Comedy", "Biography", "Documentary"]

In [ ]:
from pyspark.sql.functions import array_contains

for genre in genres:
    df = df.withColumn(genre, array_contains(col("Genre"), genre).cast("integer"))

In [ ]:
df.show()

In [ ]:
df.groupBy("Genre").count().orderBy(col("count").desc()).show()

In [9]:
print(df.printSchema())

NameError: name 'df' is not defined

+-----------+-----+
|      Genre|count|
+-----------+-----+
|      Crime|  200|
|    Romance|  200|
|   Thriller|  200|
|    Famille|  200|
|      Drama|  200|
|Documentary|  200|
|    Comédie|  200|
|Fantastique|  200|
|  Animation|  200|
|     Biopic|  200|
|     Comedy|  200|
|     Action|  200|
|     Sci-Fi|  200|
|    Musical|  199|
|     Horror|  195|
|    Western|  193|
|     Guerre|  192|
+-----------+-----+


In [5]:
import pandas as pd

a = pd.read_parquet(data_dir+"/final_dataset.parquet")
a.head()

,Title,Genre,Runtime,Actors,Rating,Summary,Type
0,Lets Fight Ghost,"Crime, Drama, Fantasy, Horror, Romance",< 30 minutes,"Lina Leandersson, K�re Hedebrant, Per Ragnar, ...",4.0,A med student with a supernatural gift tries t...,Series
1,HOW TO BUILD A GIRL,Comedy,1-2 hour,"Cleo, Paddy Considine, Beanie Feldstein, D�nal...",2.9,"When nerdy Johanna moves to London, things get...",Movie
2,The Con-Heartist,"Comedy, Romance",> 2 hrs,"Kathaleeya McIntosh, Nadech Kugimiya, Pimchano...",3.7,After her ex-boyfriend cons her out of a large...,Movie
3,Gleboka woda,Drama,< 30 minutes,"Katarzyna Maciag, Piotr Nowak, Marcin Dorocins...",3.8,A group of social welfare workers led by their...,Series
4,Only a Mother,Drama,1-2 hour,"Hugo Bj�rne, Eva Dahlbeck, Ulf Palme, Ragnar F...",3.3,An unhappily married farm worker struggling to...,Movie


In [32]:
a["Runtime"].value_counts()

Runtime
1-2 hour        2664
< 30 minutes    1512
> 2 hrs          755
1h 35min          90
1h 40min          89
                ... 
1h 02min           1
3h 30min           1
1h 00min           1
0h 52min           1
3h 39min           1
Name: count, Length: 167, dtype: int64

In [10]:
a["Runtime"][0]

'< 30 minutes'

In [29]:
def parse_runtime(runtime):
    if runtime in ['> 2 hrs', '< 30 minutes', '1-2 hour']:
        print("Already parsed")
        return runtime
    else : 
        runtime.str.replace("min", "")
    
   
    
    
        

In [30]:
#a.dropna(subset=["Runtime"], inplace=True)
A = a.apply(lambda x: parse_runtime(x["Runtime"]), axis=1)



Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Parsing
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Already parsed
Al